In [120]:
import pandas as pd
from pathlib import Path
from edisgo import flex_opt
from edisgo import network
from edisgo import tools

from edisgo import EDisGo
from edisgo.network.grids import MVGrid, LVGrid
from tabulate import tabulate
from io import StringIO

In [121]:
# Some helpful debugging functions
def pd_print(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))
    
def search_str(df,col,search_str,invert=False):
    if invert == False:
        pd_print(df.loc[df[col].str.contains(search_str)])
        return df.loc[df[col].str.contains(search_str)]
    else:
        pd_print(df.loc[~df[col].str.contains(search_str)])
        return df.loc[~df[col].str.contains(search_str)]
    
def search_str_df(df,col,search_str,invert=False):
    if invert == False:
        return df.loc[df[col].str.contains(search_str)]
    else:
        return df.loc[~df[col].str.contains(search_str)]

In [122]:
# grid_name = '1-MVLV-comm-all-0-no_sw'
grid_name = '1-MVLV-semiurb-3.202-1-no_sw'


curr_path = Path.cwd()
parent_dir = curr_path.parents[3]
# simbench_grids_dir = parent_dir / 'simbench'
grid_dir = parent_dir / grid_name
print (grid_dir)

/Users/heikernleong/Documents/localFiles/sa_code/edisgo/hk_edisgo/1-MVLV-semiurb-3.202-1-no_sw


Importing the simbench raw data into panda dataframes

In [123]:
file_list = [i for i in grid_dir.iterdir()]
simbench_dict = {i.name[:-4]:pd.read_csv(i,delimiter=";") for i in file_list}
for i in simbench_dict.keys(): print (i)
sb_ding0_dict = {}

Line
LoadProfile
RES
ExternalNet
TransformerType
StudyCases
Node
Switch
Measurement
Storage
Coordinates
StorageProfile
NodePFResult
Substation
Transformer
Load
RESProfile
LineType


## 1) buses_df

In [124]:
def label_mv_grid_id(name):
    if "LV" in name:
        return ""
    elif "MV" in name:
        return name[0:name.find(" ")]

def label_lv_grid_id(name):
    if "MV" in name:
        return ""
    elif "LV" in name:
        return name[name.find(".")+1:]
    
buses_cols_to_drop = [
    'type',
    'vmSetp',
    'vaSetp',
    'vmMin',
    'vmMax',
    'substation',
    'voltLvl'
]

buses_col_location = [
    'name',
    'x',
    'y',
    'mv_grid_id',
    'lv_grid_id',
    'v_nom',
    'in_building'
]

buses_df = simbench_dict['Node']
# buses_df_busbar = buses_df.loc[buses_df['type'].str.contains('busbar')]
# buses_df_node = buses_df.loc[buses_df['type'].str.contains('node')]
# buses_df = pd.concat([buses_df_busbar,buses_df_node],ignore_index=True)
buses_df = buses_df.rename(columns={'vmR':'v_nom'})
buses_df = buses_df.rename(columns={'id':'name'})
coord_df = simbench_dict['Coordinates']
coord_df = coord_df.set_index('id')
buses_df['x'] = buses_df['coordID'].apply(lambda coordID: coord_df.loc[coordID,'x'] )
buses_df['y'] = buses_df['coordID'].apply(lambda coordID: coord_df.loc[coordID,'y'] )
buses_df['in_building'] = 'False'

buses_df['mv_grid_id'] =  buses_df['name'].apply(label_mv_grid_id)
buses_df['lv_grid_id'] =  buses_df['subnet'].apply(label_lv_grid_id)

# get the HV grid to adopt the mv grid id
# hv_name = buses_df[buses_df['name'].str.contains('HV')]['name'].iloc[0]
hv_index = buses_df.index[buses_df['name'].str.contains('HV')]
print (hv_index)
mv_grid_id = buses_df[buses_df['name'].str.contains('MV')]['mv_grid_id'].iloc[0]
buses_df.loc[hv_index,'mv_grid_id'] = mv_grid_id 



buses_df = buses_df.drop(buses_cols_to_drop,axis=1)
buses_df = buses_df[buses_col_location]
# buses_df = buses_df.set_index('name')
pd_print(buses_df.head())
sb_ding0_dict['buses_df'] = buses_df

Int64Index([0], dtype='int64')
+----+-------------------+---------+----------+--------------+--------------+---------+---------------+
|    | name              |       x |        y | mv_grid_id   | lv_grid_id   |   v_nom | in_building   |
|----+-------------------+---------+----------+--------------+--------------+---------+---------------|
|  0 | HV1 Bus 19        | 13.489  | -14.0665 | MV2.101      |              |     110 | False         |
|  1 | MV2.101 busbar1.1 | 13.489  | -14.0665 | MV2.101      |              |      20 | False         |
|  2 | MV2.101 Bus 4     | 13.4919 | -14.0668 | MV2.101      |              |      20 | False         |
|  3 | MV2.101 Bus 5     | 13.4936 | -14.0671 | MV2.101      |              |      20 | False         |
|  4 | MV2.101 Bus 6     | 13.4951 | -14.0674 | MV2.101      |              |      20 | False         |
+----+-------------------+---------+----------+--------------+--------------+---------+---------------+


## 2) generator_df

In [125]:
gen_cols_to_drop = [
    'pRES',
    'qRES',
    'subnet',
    'voltLvl'
]

gen_col_location = [
    'name',
    'bus',
    'control',
    'p_nom',
    'type',
    'weather_cell_id',
    'subtype'
]

gen_rename_dict = {
    'id':'name',
    'node':'bus',
    'sR':'p_nom',
    'profile':'weather_cell_id'
}

slack_col_to_drop = [
    'dspf',
    'pExtNet',
    'qExtNet',
    'pWardShunt',
    'qWardShunt',
    'rXWard',
    'xXWard',
    'vmXWard',
    'subnet',
    'voltLvl'
]

def add_slack_terms(gen_df):
    add_gen_dict = {
        'p_nom':0.0,
        'type':'station',
        'weather_cell_id':"",
        'subtype':'mv_station'
    }
    for key in add_gen_dict:
        gen_df[key] = gen_df['control'].apply(lambda x: add_gen_dict[key] if x == 'Slack' else x)
    return gen_df

generators_df = simbench_dict['RES']
generators_df = generators_df.rename(columns=gen_rename_dict)
generators_df['calc_type'] = generators_df['calc_type'].apply(lambda x: x.upper())
generators_df = generators_df.rename(columns={'calc_type':'control'})
generators_df['type'] = generators_df['type'].apply(lambda x: 'solar' if x== 'PV' else x.lower())
generators_df['subtype'] = generators_df['type']
generators_df = generators_df.drop(gen_cols_to_drop, axis = 1)
generators_df = generators_df[gen_col_location]


# including the slack bus
slack_df = simbench_dict['ExternalNet']
slack_df = slack_df.rename(columns={'id':'name'})
slack_df = slack_df.rename(columns={'node':'bus'})
slack_df = slack_df.rename(columns={'calc_type':'control'})
slack_df = slack_df.drop(slack_col_to_drop,axis=1)
slack_df['control'] = slack_df['control'].apply(lambda x: 'Slack' if x == 'vavm' else x)
slack_df['name'] = slack_df['name'].apply(lambda x: x+'_slack')
slack_df = add_slack_terms(slack_df)
# pd_print(slack_df)
# pd_print (generators_df.head())
generators_df = pd.concat([slack_df,generators_df] ,ignore_index=True)
# generators_df = generators_df.set_index('name')
pd_print (generators_df.head())

sb_ding0_dict['generators_df'] = generators_df

+----+---------------------------+-------------------+-----------+---------+---------+-------------------+------------+
|    | name                      | bus               | control   |   p_nom | type    | weather_cell_id   | subtype    |
|----+---------------------------+-------------------+-----------+---------+---------+-------------------+------------|
|  0 | HV1 grid at MV2.101_slack | HV1 Bus 19        | Slack     |  0      | station |                   | mv_station |
|  1 | MV2.101 SGen 1            | MV2.101 busbar1.1 | PQ        |  1.5    | wind_mv | WP7               | wind_mv    |
|  2 | MV2.101 SGen 2            | MV2.101 Bus 4     | PQ        |  0.14   | lv_res  | lv_urban6         | lv_res     |
|  3 | MV2.101 SGen 3            | MV2.101 Bus 5     | PQ        |  0.1818 | lv_res  | lv_semiurb5       | lv_res     |
|  4 | MV2.101 SGen 4            | MV2.101 Bus 6     | PQ        |  0.1818 | lv_res  | lv_semiurb5       | lv_res     |
+----+---------------------------+------

## 3) lines_df

In [126]:
lines_cols_to_drop = [
    'loadingMax',
    'subnet',
    'voltLvl'
]

lines_col_location = [
    'name',
    'bus0',
    'bus1',
    'length',
    'r',
    'x',
    's_nom',
    'num_parallel',
    'kind',
    'type_info'
]

def get_line_char(lines_df, line_type_df):
    char_dict = {
        'r':'r',
        'x': 'x',
        'iMax':'iMax',
        'kind':'type'
    }
    for key in char_dict:
        lines_df[key] = lines_df['type_info'].apply(lambda x: line_type_df.loc[x,char_dict[key]])
    return lines_df

def cal_s_nom(name):
    x = name['x']
    r = name['r']
    i = name['iMax']
    return ((((x**2 + r**2))**0.5) * (i)**2)/(10**6)

def mul_length(name):
    return name['x']*name['length']

lines_df = simbench_dict['Line']
line_type_df = simbench_dict['LineType']
line_type_df = line_type_df.set_index('id')

# pd_print(line_type_df.head())
lines_rename_dict = {
    'id':'name',
    'nodeA':'bus0',
    'nodeB':'bus1',
    'type': 'type_info'
}
lines_df = lines_df.rename(columns=lines_rename_dict)
lines_df = lines_df.drop(lines_cols_to_drop,axis=1)
lines_df = get_line_char(lines_df,line_type_df)
lines_df['kind'] = lines_df['kind'].apply(lambda x : 'line' if x =='ohl' else x)
lines_df['num_parallel'] = 1
lines_df['r'] = lines_df.apply(lambda name: name['r']*name['length'],axis=1)
lines_df['x'] = lines_df.apply(lambda name: name['x']*name['length'],axis=1)
lines_df['s_nom'] = lines_df.apply(cal_s_nom,axis=1)
lines_df = lines_df[lines_col_location]
# lines_df = lines_df.set_index('name')
pd_print(lines_df.head())

sb_ding0_dict['lines_df'] = lines_df

+----+----------------+-------------------+---------------+----------+---------+---------+------------+----------------+--------+-----------------------------+
|    | name           | bus0              | bus1          |   length |       r |       x |      s_nom |   num_parallel | kind   | type_info                   |
|----+----------------+-------------------+---------------+----------+---------+---------+------------+----------------+--------+-----------------------------|
|  0 | MV2.101 Line 1 | MV2.101 busbar1.1 | MV2.101 Bus 4 |     0.7  | 0.2191  | 0.0924  | 0.0151004  |              1 | cable  | NA2XS2Y 1x95 RM/25 12/20 kV |
|  1 | MV2.101 Line 2 | MV2.101 Bus 4     | MV2.101 Bus 5 |     0.45 | 0.14085 | 0.0594  | 0.00970741 |              1 | cable  | NA2XS2Y 1x95 RM/25 12/20 kV |
|  2 | MV2.101 Line 3 | MV2.101 Bus 5     | MV2.101 Bus 6 |     0.4  | 0.1252  | 0.0528  | 0.00862881 |              1 | cable  | NA2XS2Y 1x95 RM/25 12/20 kV |
|  3 | MV2.101 Line 4 | MV2.101 Bus 6   

## 4) loads_df

In [131]:
loads_rename_dict = {
    'id':'name',
    'node':'bus',
    'profile': 'sector'
}

loads_cols_to_drop = [
    'pLoad',
    'qLoad',
    'subnet',
    'voltLvl',
    'sR'
]

loads_cols_location = [
    'name',
    'bus',
    'peak_load',
    'annual_consumption',
    'sector'
]

def cal_annual_consumption(name):
    sR = name['sR']
    sector = name['sector']
    loads_profile[sector+'_sload'] = (loads_profile[ sector + '_pload']**2+loads_profile[sector + '_qload']**2)**0.5
    annual_consumption = loads_profile[sector+'_sload'].sum()*sR
    return annual_consumption

def cal_peak_load(name):
    sR = name['sR']
    sector = name['sector']
    peak_load = loads_profile[sector+'_pload'].max()*sR
    return peak_load

loads_profile = simbench_dict['LoadProfile']
loads_df = simbench_dict['Load']
loads_df = loads_df.rename(columns=loads_rename_dict)
loads_df['peak_load'] = loads_df.apply(cal_peak_load,axis=1)
loads_df['annual_consumption'] = loads_df.apply(cal_annual_consumption,axis=1)
loads_df = loads_df[loads_cols_location]
# loads_df = loads_df.set_index('name')
pd_print(loads_df.head())

sb_ding0_dict['loads_df'] = loads_df

+----+------------------+-------------------+-------------+----------------------+-------------+
|    | name             | bus               |   peak_load |   annual_consumption | sector      |
|----+------------------+-------------------+-------------+----------------------+-------------|
|  0 | HV1_MV2.101_load | MV2.101 busbar1.1 |    0.3641   |              6912.21 | G2-A        |
|  1 | MV2.101 Load 2   | MV2.101 Bus 4     |    0.206082 |              2704.23 | lv_urban6   |
|  2 | MV2.101 Load 3   | MV2.101 Bus 5     |    0.213884 |              2659.97 | lv_semiurb5 |
|  3 | MV2.101 Load 4   | MV2.101 Bus 6     |    0.213884 |              2659.97 | lv_semiurb5 |
|  4 | MV2.101 Load 5   | MV2.101 Bus 7     |    0.130194 |              2003.56 | lv_semiurb4 |
+----+------------------+-------------------+-------------+----------------------+-------------+


## 5) switches_df
currently not implemented due to different means of working on the switches between the 2 systems

In [128]:
switch_rename_dict = {
    'id':'name',
    'nodeA':'bus_closed',
    'nodeB':'bus_open',
    'substation':'branch',
    'type':'type_info'
}

switch_col_location = [
    'name',
    'bus_closed',
    'bus_open',
    'branch',
    'type_info'
]

switches_df = simbench_dict['Switch']
switches_df = switches_df.rename(columns=switch_rename_dict)
switches_df = switches_df[switch_col_location]

pd_print(switches_df)

sb_ding0_dict['switches_df'] = switches_df

+----+------------------------------+-----------------+-------------------+----------+-------------+
|    | name                         | bus_closed      | bus_open          |   branch | type_info   |
|----+------------------------------+-----------------+-------------------+----------+-------------|
|  0 | MV2.101 loop_line_switch 1.2 | MV2.101 Bus 57  | MV2.101 Bus 57_1  |      nan | LBS         |
|  1 | MV2.101 loop_line_switch 2.2 | MV2.101 Bus 116 | MV2.101 Bus 116_1 |      nan | LBS         |
|  2 | MV2.101 loop_line_switch 3.2 | MV2.101 Bus 13  | MV2.101 Bus 13_1  |      nan | LBS         |
|  3 | MV2.101 loop_line_switch 4.2 | MV2.101 Bus 86  | MV2.101 Bus 86_1  |      nan | LBS         |
|  4 | MV2.101 loop_line_switch 5.2 | MV2.101 Bus 75  | MV2.101 Bus 75_1  |      nan | LBS         |
|  5 | MV2.101 loop_line_switch 6.2 | MV2.101 Bus 25  | MV2.101 Bus 25_1  |      nan | LBS         |
|  6 | MV2.101 loop_line_switch 7.2 | MV2.101 Bus 37  | MV2.101 Bus 37_1  |      nan | LBS 

## 6) transformer_df

In [129]:
trans_rename_dict = {
    'id':'name',
    'nodeHV':'bus0',
    'nodeLV':'bus1',
}

trans_col_location = [
    'name',
    'bus0',
    'bus1',
    's_nom',
    'r',
    'x',
    'type',
    'type_info'
]

sb_trans_df = simbench_dict['Transformer']
sb_trans_type_df = simbench_dict['TransformerType']
sb_trans_df = sb_trans_df.rename(columns=trans_rename_dict)
sb_trans_type_df = sb_trans_type_df.set_index('id')
sb_trans_df['s_nom'] = sb_trans_df['type'].apply(lambda x: sb_trans_type_df.loc[x,'sR'] )
sb_trans_df['r'] = ''
sb_trans_df['x'] = ''
sb_trans_df['type_info'] = sb_trans_df['type']
sb_trans_df = sb_trans_df[trans_col_location]

transformers_hvmv_df = sb_trans_df[sb_trans_df['bus0'].str.contains('HV')]
transformers_hvmv_df = sb_trans_df[sb_trans_df['bus1'].str.contains('MV')]

transformers_df = sb_trans_df[sb_trans_df['bus0'].str.contains('MV')]
transformers_df = sb_trans_df[sb_trans_df['bus1'].str.contains('LV')]

# transformers_hvmv_df = transformers_hvmv_df.set_index('name')
# transformers_df = transformers_df.set_index('name')

pd_print(transformers_hvmv_df.head())
pd_print(transformers_df.head())

sb_ding0_dict['transformers_hvmv_df'] = transformers_hvmv_df
sb_ding0_dict['transformers_df'] = transformers_df

+----+--------------------+------------+-------------------+---------+-----+-----+-----------------------+-----------------------+
|    | name               | bus0       | bus1              |   s_nom | r   | x   | type                  | type_info             |
|----+--------------------+------------+-------------------+---------+-----+-----+-----------------------+-----------------------|
|  0 | HV1-MV2.101-Trafo1 | HV1 Bus 19 | MV2.101 busbar1.1 |      40 |     |     | 40 MVA 110/20 kV YNd5 | 40 MVA 110/20 kV YNd5 |
|  1 | HV1-MV2.101-Trafo2 | HV1 Bus 19 | MV2.101 busbar1.1 |      40 |     |     | 40 MVA 110/20 kV YNd5 | 40 MVA 110/20 kV YNd5 |
+----+--------------------+------------+-------------------+---------+-----+-----+-----------------------+-----------------------+
+----+-----------------------+----------------+----------------+---------+-----+-----+-----------------------------+-----------------------------+
|    | name                  | bus0           | bus1           |   

## Debugging code

In [130]:
for key in sb_ding0_dict:
    print (key)

buses_df
generators_df
lines_df
loads_df
switches_df
transformers_hvmv_df
transformers_df


In [77]:
# for key in sb_ding0_dict:
#     print ("key: " + key)
#     if 'generators' or 'loads' in key:
#         search_str(sb_ding0_dict[key],'bus','HV1 Bus 13',invert=False)
#     elif 'lines' or 'transformers' in key:
#         search_str(sb_ding0_dict[key],'bus0','HV1 Bus 13',invert=False)
#         search_str(sb_ding0_dict[key],'bus1','HV1 Bus 13',invert=False)
#     elif 'switches' in key:
#         search_str(sb_ding0_dict[key],'bus_closed','HV1 Bus 13',invert=False)
#         search_str(sb_ding0_dict[key],'bus_open','HV1 Bus 13',invert=False)

# search_str(generators_df,'name','MV4.101 MV SGen 1',invert=False)
search_str(buses_df,'name','HV',invert=False)
# sb_node_df = simbench_dict['Node']
# pd_print(sb_node_df.head())
# search_str(sb_node_df,'id','MV4.101 Bus 28',invert=False)
# search_str(sb_node_df,'type','busbar',invert=True)
# search_str(sb_node_df,'id','HV1 Bus 13',invert=False)

# ding0_grid = parent_dir / 'Data_working' / 'ding0_test_network'
# ding0_bus_df_csv = ding0_grid / 'buses.csv'
# ding0_bus_df = pd.read_csv(ding0_bus_df_csv)
# pd_print (ding0_bus_df.head())
# search_str(ding0_bus_df,'name','hv',invert=False)

+------+------------+---------+---------+--------------+--------------+---------+---------------+
|      | name       |       x |       y | mv_grid_id   | lv_grid_id   |   v_nom | in_building   |
|------+------------+---------+---------+--------------+--------------+---------+---------------|
| 6099 | HV1 Bus 13 | 11.4511 | 53.5923 | MV4.101      |              |     110 | False         |
+------+------------+---------+---------+--------------+--------------+---------+---------------+


## EDisGo direct import testing

In [117]:
edisgo_obj = EDisGo(import_timeseries=False)
# importing the dataframes into the edisgo_obj
# for key in sb_ding0_dict:
#     print(key)
#     edisgo_obj.topology.key = sb_ding0_dict[key]
    
edisgo_obj.topology.buses_df = buses_df
edisgo_obj.topology.generators_df = generators_df
edisgo_obj.topology.lines_df = lines_df
edisgo_obj.topology.loads_df = loads_df
edisgo_obj.topology.switches_df = switches_df
edisgo_obj.topology.transformers_hvmv_df = transformers_hvmv_df
edisgo_obj.topology.transformers_df = transformers_df

# creating the mv grid
edisgo_obj.topology.mv_grid = MVGrid(id=mv_grid_id, edisgo_obj=edisgo_obj)
edisgo_obj.topology._grids = {}
edisgo_obj.topology._grids[str(edisgo_obj.topology.mv_grid)] = edisgo_obj.topology.mv_grid

#creating the lv grid
lv_grid_df =  search_str_df(buses_df,'name','LV',invert=False)
# pd_print (lv_grid_df)
lv_grid_ids = lv_grid_df['lv_grid_id'].unique()
for lv_grid_id in lv_grid_ids:
        lv_grid = LVGrid(id=lv_grid_id, edisgo_obj=edisgo_obj)
        edisgo_obj.topology.mv_grid._lv_grids.append(lv_grid)
        edisgo_obj.topology._grids[str(lv_grid)] = lv_grid

In [141]:
# print(edisgo_obj.__dict__)
# print(edisgo_obj.topology.__dict__)
# edisgo_obj.to_graph()
# edisgo_obj.plot_mv_grid_topology(technologies=True)
edisgo_obj.analyze()

AttributeError: 'Topology' object has no attribute '_storage_units_df'

## Test Export to csv

In [79]:
sb_ding0_dir = parent_dir / 'sb_ding0'
print (sb_ding0_dir)

if not sb_ding0_dir.exists():
    Path.mkdir(sb_ding0_dir)
    
print (sb_ding0_dict.keys())    
    
for key in sb_ding0_dict:
    sb_ding0_dict[key] = sb_ding0_dict[key].set_index('name')
    sb_ding0_dict[key].to_csv(str(sb_ding0_dir)+'/'+ key[:-3] +'.csv')

/Users/heikernleong/Documents/localFiles/sa_code/edisgo/hk_edisgo/sb_ding0
dict_keys(['buses_df', 'generators_df', 'lines_df', 'loads_df', 'switches_df', 'transformers_hvmv_df', 'transformers_df'])
